In [3]:
import cv2
import os
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
from natsort import natsorted  # frame_0001, frame_0010 순서 보장용

def analyze_frame_folders_no_fps(
    root_dir: str,
    openclose_model_path: str = "best_openclose.pt",
    fullempty_model_path: str = "best_fullempty.pt",
):
    """
    test_video/ 아래 있는 모든 프레임 폴더를 순회하며,
    프레임별 YOLO 상태(open/close, full/empty)를 CSV로 저장.

    출력 경로:
        root_dir/out_yolo/{폴더명}_yolo_states.csv

    CSV 컬럼:
        video_name, frame_idx, frame_name,
        box_count,
        open_count, closed_count,
        full_count, empty_count
    """
    root = Path(root_dir)
    yolo_out_root = root / "out_yolo"
    yolo_out_root.mkdir(exist_ok=True)

    # 1) 모델 로드
    model_openclose = YOLO(openclose_model_path)
    model_fullempty = YOLO(fullempty_model_path)

    print("[INFO] open/close model classes:", model_openclose.names)
    print("[INFO] full/empty model classes:", model_fullempty.names)

    # 👉 여기서 네가 학습한 클래스 ID에 맞게 수정해야 함!
    # 예시:
    #   best_openclose.pt: 0 = open_box, 1 = closed_box
    #   best_fullempty.pt: 0 = full_box, 1 = empty_box
    OPEN_IDS   = {0}
    CLOSED_IDS = {1}
    FULL_IDS   = {0}
    EMPTY_IDS  = {1}

    # 2) 루트 아래 폴더들 탐색
    folder_list = [f for f in root.iterdir() if f.is_dir() and f.name != "out_yolo"]
    print(f"[INFO] Found {len(folder_list)} video folders under {root_dir}")

    for folder in folder_list:
        video_name = folder.name
        print(f"\n[PROCESS] {video_name}")

        # 3) 프레임 이미지 파일 리스트 (자연스러운 순서로 정렬)
        img_list = natsorted([
            p for p in folder.iterdir()
            if p.suffix.lower() in [".jpg", ".png"]
        ])

        if len(img_list) == 0:
            print(f"[WARN] {video_name} has no images. skip.")
            continue

        results = []

        for idx, img_path in enumerate(img_list):
            frame = cv2.imread(str(img_path))
            if frame is None:
                print(f"[WARN] Failed to read: {img_path}")
                continue

            # 4) YOLO 추론 (open/close)
            res_oc = model_openclose(frame, verbose=False)[0]
            # 5) YOLO 추론 (full/empty)
            res_fe = model_fullempty(frame, verbose=False)[0]

            # 각 결과에서 class id 뽑기
            cls_oc = res_oc.boxes.cls.cpu().numpy().astype(int) if res_oc.boxes is not None else []
            cls_fe = res_fe.boxes.cls.cpu().numpy().astype(int) if res_fe.boxes is not None else []

            # 집계
            open_count   = sum(c in OPEN_IDS   for c in cls_oc)
            closed_count = sum(c in CLOSED_IDS for c in cls_oc)

            full_count   = sum(c in FULL_IDS   for c in cls_fe)
            empty_count  = sum(c in EMPTY_IDS  for c in cls_fe)

            # 상자 총 개수는 open/close 모델 탐지 수 기준
            box_count = len(cls_oc)

            results.append({
                "video_name": video_name,
                "frame_idx": idx,                 # 0부터 시작
                "frame_name": img_path.name,      # frame_000123.jpg 같은 이름
                "box_count": box_count,
                "open_count": open_count,
                "closed_count": closed_count,
                "full_count": full_count,
                "empty_count": empty_count,
            })

        # 6) CSV 저장
        out_csv = yolo_out_root / f"{video_name}_yolo_states.csv"
        df = pd.DataFrame(results)
        df.to_csv(out_csv, index=False, encoding="utf-8-sig")

        print(f"[SAVE] {out_csv} ({len(df)} rows)")

    print("\n[INFO] All folders processed.")


if __name__ == "__main__":
    analyze_frame_folders_no_fps(
        root_dir="test_video",
        openclose_model_path="best_openclose.pt",
        fullempty_model_path="best_fullempty.pt",
    )


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\user\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[INFO] open/close model classes: {0: 'open', 1: 'close'}
[INFO] full/empty model classes: {0: 'empty', 1: 'full'}
[INFO] Found 10 video folders under test_video

[PROCESS] video_ idle_001
[SAVE] test_video\out_yolo\video_ idle_001_yolo_states.csv (255 rows)

[PROCESS] video_missing1_new_001
[SAVE] test_video\out_yolo\video_missing1_new_001_yolo_states.csv (187 rows)

[PROCESS] video_missing1_new_002
[SAVE] test_video\out_yolo\video_missing1_new_002_yolo_states.csv (216 rows)

[PROCESS] video_missing1_new_003
[SAVE] test_video\out_yolo\video_missing1_new_003_yolo_states.csv (279 rows)

[PROCESS] video_missing2_new_001
[SAVE] test_video\out_yolo\video_missing2_new_001

## 시각화

In [4]:
import cv2
import os
from pathlib import Path
from ultralytics import YOLO

def visualize_frames_with_yolo(
    frames_dir: str,
    openclose_model_path: str = "best_openclose.pt",
    fullempty_model_path: str = "best_fullempty.pt",
):
    """
    frames_dir: 프레임 이미지가 들어있는 폴더 (예: test_video/video_normal_001)
    - A / 왼쪽 화살표 : 이전 프레임
    - D / 오른쪽 화살표 : 다음 프레임
    - Q : 종료
    """

    frames_dir = Path(frames_dir)

    # 이미지 파일 리스트 (frame_000000.jpg 형태라면 기본 sorted도 괜찮음)
    img_paths = sorted([
        p for p in frames_dir.iterdir()
        if p.suffix.lower() in [".jpg", ".png", ".jpeg"]
    ])

    if not img_paths:
        print(f"[ERROR] No images found in {frames_dir}")
        return

    print(f"[INFO] {frames_dir.name}: {len(img_paths)} frames found.")

    # 모델 로드
    model_openclose = YOLO(openclose_model_path)
    model_fullempty = YOLO(fullempty_model_path)

    print("[INFO] open/close model classes:", model_openclose.names)
    print("[INFO] full/empty model classes:", model_fullempty.names)

    # 👉 네가 학습한 클래스 ID에 맞게 수정
    # 예시:
    #   best_openclose.pt : 0=open_box, 1=closed_box
    #   best_fullempty.pt : 0=full_box, 1=empty_box
    OPEN_IDS   = {0}
    CLOSED_IDS = {1}
    FULL_IDS   = {0}
    EMPTY_IDS  = {1}

    # BGR 색상 설정
    color_open   = (0, 255, 0)     # 초록
    color_closed = (0, 0, 255)     # 빨강
    color_full   = (255, 0, 0)     # 파랑
    color_empty  = (0, 255, 255)   # 노랑

    idx = 0
    num_frames = len(img_paths)

    while True:
        img_path = img_paths[idx]
        frame = cv2.imread(str(img_path))
        if frame is None:
            print(f"[WARN] Failed to read image: {img_path}")
            # 다음 프레임으로
            idx = (idx + 1) % num_frames
            continue

        # YOLO 추론
        res_oc = model_openclose(frame, verbose=False)[0]
        res_fe = model_fullempty(frame, verbose=False)[0]

        # 카운트용
        open_count = closed_count = full_count = empty_count = 0

        # --- open/close bbox 그리기 ---
        if res_oc.boxes is not None and len(res_oc.boxes) > 0:
            boxes = res_oc.boxes.xyxy.cpu().numpy()
            classes = res_oc.boxes.cls.cpu().numpy().astype(int)

            for box, cls_id in zip(boxes, classes):
                x1, y1, x2, y2 = box.astype(int)

                if cls_id in OPEN_IDS:
                    color = color_open
                    label = "OPEN"
                    open_count += 1
                elif cls_id in CLOSED_IDS:
                    color = color_closed
                    label = "CLOSED"
                    closed_count += 1
                else:
                    # 혹시 정의 안한 클래스
                    color = (200, 200, 200)
                    label = f"OC_{cls_id}"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # --- full/empty bbox 그리기 ---
        if res_fe.boxes is not None and len(res_fe.boxes) > 0:
            boxes = res_fe.boxes.xyxy.cpu().numpy()
            classes = res_fe.boxes.cls.cpu().numpy().astype(int)

            for box, cls_id in zip(boxes, classes):
                x1, y1, x2, y2 = box.astype(int)

                if cls_id in FULL_IDS:
                    color = color_full
                    label = "FULL"
                    full_count += 1
                elif cls_id in EMPTY_IDS:
                    color = color_empty
                    label = "EMPTY"
                    empty_count += 1
                else:
                    color = (150, 150, 150)
                    label = f"FE_{cls_id}"

                # thickness를 다르게 해서 서로 구분되게
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 1)
                cv2.putText(frame, label, (x1, y2 + 15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # 상단에 요약 정보 텍스트
        info_text = (
            f"{frames_dir.name} | frame {idx+1}/{num_frames} "
            f"| Open:{open_count} Closed:{closed_count} "
            f"| Full:{full_count} Empty:{empty_count}"
        )
        cv2.putText(frame, info_text, (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("YOLO Frame Viewer", frame)

        # 키 입력 대기 (무한 대기)
        key = cv2.waitKey(0) & 0xFF

        # Q or ESC : 종료
        if key in [ord('q'), 27]:
            break

        # D 또는 → : 다음 프레임
        elif key in [ord('d'), 83]:  # 83: 오른쪽 화살표
            idx = min(idx + 1, num_frames - 1)

        # A 또는 ← : 이전 프레임
        elif key in [ord('a'), 81]:  # 81: 왼쪽 화살표
            idx = max(idx - 1, 0)

        # 그 외 키: 그냥 다시 같은 프레임 유지

    cv2.destroyAllWindows()


if __name__ == "__main__":
    # 예시: test_video 아래 특정 세트
    frames_dir = r"test_video\video_normal_new_001(normal)"
    visualize_frames_with_yolo(
        frames_dir,
        openclose_model_path="best_openclose.pt",
        fullempty_model_path="best_fullempty.pt",
    )


[INFO] video_normal_new_001(normal): 317 frames found.
[INFO] open/close model classes: {0: 'open', 1: 'close'}
[INFO] full/empty model classes: {0: 'empty', 1: 'full'}
